In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import GRU,LSTM,Input,Conv1D, SpatialDropout1D,Dense, BatchNormalization, Activation, MaxPooling1D, GlobalAveragePooling1D, Add,Flatten,Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping

### Load Training Data

In [2]:
with open('Train_X.npy', 'rb') as f:
    Train_X = np.load(f,allow_pickle=True)

with open('Train_y.npy', 'rb') as f:
    Train_y = np.load(f,allow_pickle=True)

with open('Test_X.npy', 'rb') as f:
    Test_X = np.load(f,allow_pickle=True)
    
with open('Test_y.npy', 'rb') as f:
    Test_y = np.load(f,allow_pickle=True)

print(Train_X.shape,Train_y.shape,Test_X.shape,Test_y.shape)

(1600000, 256) (1600000,) (498, 256) (498,)


In [3]:
Train_y[Train_y == 4] = 1
Test_y[Test_y == 4] = 1

In [12]:
def get_model():
    model = Sequential()
    model.add(GRU(units=4, input_shape=[256,1]))
    model.add(Dense(4))
    model.add(Dense(1))
    return model

def step_decay(epoch):
    """
    a function to decay the learning rate 0.94 every 2 epoch
    """
    initial_lrate = 0.003
    drop = 0.95
    epochs_drop = 2
    end_lr = 0.00001
    lrate = initial_lrate * np.power(drop,  
        np.floor((1+epoch)/epochs_drop))
    if lrate > end_lr:
        return lrate
    else:
        return end_lr
lr_scheduler = LearningRateScheduler(step_decay)
el = EarlyStopping(monitor='val_loss', patience=8)

#parameter setting
epochs = 10
batch_size = 128
optimizer = Adam(0.003)

#model compile
model = get_model()
model.compile(optimizer,loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(np.expand_dims(Train_X,axis=2),Train_y,epochs = epochs, 
                                      batch_size = batch_size,validation_data=(np.expand_dims(Test_X,axis=2),Test_y),
                                      callbacks=[lr_scheduler,el],verbose = 1)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 4)                 84        
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 109
Trainable params: 109
Non-trainable params: 0
_________________________________________________________________
Train on 1600000 samples, validate on 498 samples
Epoch 1/10
1600000/1600000 [==============================] - 673s 421us/sample - loss: 0.6927 - accuracy: 0.5139 - val_loss: 0.7395 - val_accuracy: 0.35
Epoch 2/10
1600000/1600000 [==============================] - 674s 421us/sample - loss: 0.6835 - accuracy: 0.5372 - val_loss: 0.6351 - val_accuracy: 0.39
Epoch 3/10
1600000/1600000 [